# Assignment 3

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- sklearn [link](https://scikit-learn.org/)
- Keras [link](https://keras.io/guides/)

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q3.ipynb`.


## Question 3
Fake news is a widespread problem and there are many methods for combating it.
You have to build a fake news detection system using a ML model. Train any ML model (ANN, LSTM) over the given Dataset.
The dataset has short statements spoken by people and has the meta-information and corresponding label for those sentences. 
Your target is label column which has 6 labels(in the increasing order of truthfullness): pants-fire, false, barely-true, half-true, mostly-true, true.

The features are 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c', 'venue' and the target is column "label".

The statement is made by speaker whose job, party are given along with 6 columns which are an account of the  type of news(labels) the person has shared before. 
The person who has shared fake content before is likely to share it in future and this can be accounted by the ML model as a feature. Column barely_true_c contains how many barely_true news has the speaker shared (and so is with column X_c, value of X_c is number of X the person shared).


You have to perform two tasks:
* task1: Binary classification <br>
Classify the given news as true/false. Take the labels pants-fire, false, barely-true as false and rest (half-true, mostly-true, true) as true.
* task2: Six-way classification <br>
Classify the given news into six-classes "pants-fire, false, barely-true, half-true, mostly-true, true".

For each of the tasks:
1) Experiment with depth of network and try to fine-tune hyperparameters reporting your observations. <br>
2) Report the accuracy, f1-score, confusion matrix on train, val and test sets. <br>
3) Experiment with bag-of-words, glove and bert embeddings(code given in the below notebook) and report results. <br> Comment on what is the affect of embedding on the results.

The pre-processing code is provided, you need to write the training and test.

Note: You are supposed to train on trainset, fine-tune on val and just eval on test set. If found that you trained on val/test sets, the penalty will be incurred.

In [1]:
!pip install numpy
!pip install tensorflow
!pip install re
!pip install nltk
!pip install keras
!pip install sklearn

     |████████████████████████████████| 462 kB 5.5 MB/s 
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/com

In [2]:
!pip install -U keras

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/main_parser.py", line 8, in <module>
    from pip._internal.cli import cmdoptions
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/cmdoptions.py", line 23, in <module>
    from pip._internal.cli.parser import ConfigOptionParser
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/parser.py", line 12, in <module>
    from pip._internal.configuration import Configuration, ConfigurationError
  File "/usr/local/lib/python3.7/dist-packages/pip/_intern

In [3]:
!which keras

In [4]:
# Importing libraries
import pandas as pd
# from tensorflow import keras  #feel free to use any other library
import numpy as np

import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.optimizers import RMSprop, SGD
from keras.utils import np_utils
from keras.layers import Bidirectional, Flatten, LSTM, Dense, Dropout, Activation
from keras.models import Model, Sequential
# from keras.optimizers import SGD, Adam

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd drive/MyDrive/SMAI/Assignment3/

/content/drive/MyDrive/SMAI/Assignment3


In [7]:
%ls

glove/        q4_data/                         rollnumber_Assignment3_Q3.ipynb
glove.6B.zip  rollnumber_Assignment3_Q1.ipynb  rollnumber_Assignment3_Q4.ipynb
q3_data/      rollnumber_Assignment3_Q2.ipynb


In [8]:
train = pd.read_csv('q3_data/train.csv')
val = pd.read_csv('q3_data/val.csv')
test = pd.read_csv('q3_data/test.csv')

In [9]:
# Dropping the 'id' column
train.drop('id', axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)
val.drop('id', axis = 1, inplace = True)

In [10]:
train.head(5)

,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c,venue
0,False,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0,0,0,a mailer
1,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70,71,160,163,9,Denver
3,False,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7,19,3,5,44,a news release
4,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15,9,20,19,2,an interview on CNN


In [11]:
# Checking the shape of data
print(train.shape)
print(val.shape)
print(test.shape)

(10269, 13)
(1284, 13)
(1283, 13)


## Clean and pre-process data
* Replace missing values
* Remove numbers and special characters
* Convert to upper-case

We experiment with two types of processing, one directly appending the other attributes like subject, job, state, party to sentence and then applying bag of words on it.

Other being encoding sentence with glove embeddings and passing just that.

In [12]:

def dataPreprocessing(data):
    '''Function for cleaning the dataset
    '''
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = statement.split()
        
        # you can experiment with any other stemmers
        ps = PorterStemmer()
        statement = [ps.stem(word) for word in statement if not word in set(stopwords.words('english'))] # Stemming the dataset and removing stopwords
        statement = ' '.join(statement)
        subject = data['subject'][x].replace(',', ' ')
        speaker = data['speaker'][x]
        job = data['job'][x].lower()
        # job = job.replace(' ', '-')
        state = data['state'][x].lower()
        party = data['party'][x].lower()
        corpus.append(statement + ' '  + subject + ' ' + job + ' ' + state + ' ' + party)
    return corpus

In [13]:
x_train = dataPreprocessing(train)
x_val = dataPreprocessing(val) 
x_test = dataPreprocessing(test) 

In [14]:
len(x_train), len(x_val), len(x_test)

(10269, 1284, 1283)

In [15]:
corpus = x_train + x_val + x_test
len(corpus)

12836

## Using bag-of-words embedding


In [16]:
# Converting the corpus into bag-of-words
cv = CountVectorizer(max_features = 8000)
X = cv.fit_transform(corpus).toarray()

In [17]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [18]:
X.shape

(12836, 8000)

In [19]:
train.columns

Index(['label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
       'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c', 'venue'],
      dtype='object')

In [20]:
# Selecting the columns 'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c'
label_cols = ['barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c']
x_train2 = train[label_cols]
x_val2 = val[label_cols]
x_test2 = test[label_cols]

In [21]:
x_train2.head()
X[:len(x_train)]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [22]:
# Stacking x_train and x_train2 horizontally
x_train_bow = np.hstack((X[:len(x_train)], x_train2))
x_val_bow = np.hstack((X[len(x_train):len(x_train)+len(x_val)], x_val2))
x_test_bow = np.hstack((X[len(x_train)+len(x_val):], x_test2))

In [23]:
x_train_bow.shape

(10269, 8005)

## Use of Glove Embedding


download glove embeddings from 'https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip'
and place in your current working folder


In [27]:
!unzip "glove.6B.zip" -d "glove"

Archive:  glove.6B.zip
replace glove/glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [29]:
emmbed_dict = {}
with open('glove/glove.6B.200d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector

In [30]:
emmbed_dict['oov'] = np.zeros(200)

In [31]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def dataPreprocessing_glove(data):
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = word_tokenize(statement)

        embed_statement = []
        for w in statement:
            if w in emmbed_dict:
                embed_statement.append(emmbed_dict[w])
            else:
                embed_statement.append(emmbed_dict['oov'])
         
        # bonus: Think how you can encode the below features(hint: look upon label encoding or training your own word2vec or any other embedding model)
    
#         subject = data['subject'][x].replace(',', ' ')
#         speaker = data['speaker'][x]
#         job = data['job'][x].lower()
#         # job = job.replace(' ', '-')
#         state = data['state'][x].lower()
#         party = data['party'][x].lower()
        corpus.append(embed_statement)
    
    return np.array(corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [32]:
x_train_glove = dataPreprocessing_glove(train)
x_val_glove = dataPreprocessing_glove(val) 
x_test_glove = dataPreprocessing_glove(test) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [33]:
def mean_pooling(x_glove):
  averaged_emb = []
  for x in x_glove:
    averaged_emb.append(np.mean(x, axis=0))
  return np.array(averaged_emb)

In [34]:
x_train_glove = mean_pooling(x_train_glove)
x_val_glove = mean_pooling(x_val_glove)
x_test_glove = mean_pooling(x_test_glove)

In [35]:
x_train_glove.shape

(10269, 200)

In [36]:
x_train_glove = np.hstack((x_train_glove, x_train2))
x_val_glove = np.hstack((x_val_glove, x_val2))
x_test_glove = np.hstack((x_test_glove, x_test2))

In [37]:
x_train_glove.shape

(10269, 205)

## Use of bert embeddings
note: we used our pre-processed code for bow which has the attributed appended to end the end of sentence. 

In [45]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 3.6 MB/s 
     |████████████████████████████████| 4.0 MB 12.0 MB/s 
     |████████████████████████████████| 1.2 MB 41.0 MB/s 
     |████████████████████████████████| 77 kB 7.3 MB/s 
     |████████████████████████████████| 6.6 MB 35.9 MB/s 
     |████████████████████████████████| 596 kB 53.0 MB/s 
     |████████████████████████████████| 895 kB 43.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=da7475773623aa84f50848e1cd6d3028fa275a24d4b120c412489bf6e245f5bd
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [46]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

x_train_bert = np.hstack((model.encode(x_train), x_train2))
x_val_bert = np.hstack((model.encode(x_val), x_val2))
x_test_bert = np.hstack((model.encode(x_test), x_test2))

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Now use the above 3 types of embedded inputs(bow, glove, bert embeddings) for the 2 classification tasks and compare their outputs


# Six-way classification

## Preprocessing

In [54]:
num_classes = 6
# Preprocessing function for the labels
def categorize(data):
    y = data["label"].tolist()

    # Encoding the Dependent Variable
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)

    # Converting to binary class matrix
    y = np_utils.to_categorical(y, num_classes)
    return y

In [55]:
y_train_six_way = categorize(train)
y_test_six_way = categorize(test)
y_val_six_way = categorize(val)

Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


## Model

### BOW

In [56]:
## write your code here
# Initialize hyperparameters
learning_rate = 1e-3
epochs = 10
batch_size = 500
num_classes = 6
# train
# test
# report accuracy, f1-score and confusion matrix

In [57]:
def create_model():
  # Creating the model
  model = Sequential()
  model.add(Dense(8005, activation = 'relu', kernel_initializer = 'glorot_uniform'))
  model.add(Dropout(0.80))
  model.add(Dense(200, activation = 'relu'))
  model.add(Dropout(0.65))
  model.add(Dense(num_classes, activation = 'softmax'))
  rmsprop = RMSprop(learning_rate)
  model.compile(loss = 'categorical_crossentropy', optimizer = rmsprop, metrics = ['accuracy']) # Compile model
  return model

In [58]:
# Model fitting
model = create_model()
model.fit(x_train_bow, y_train_six_way, batch_size = batch_size, epochs = epochs, verbose = 1, validation_data = (x_val_bow, y_val_six_way))

Epoch 1/10
21/21 [==============================] - 6s 131ms/step - loss: 2.0676 - accuracy: 0.2642 - val_loss: 1.5661 - val_accuracy: 0.4252
Epoch 2/10
21/21 [==============================] - 2s 114ms/step - loss: 1.5544 - accuracy: 0.3823 - val_loss: 1.4585 - val_accuracy: 0.4221
Epoch 3/10
21/21 [==============================] - 2s 114ms/step - loss: 1.4211 - accuracy: 0.4272 - val_loss: 1.4401 - val_accuracy: 0.3995
Epoch 4/10
21/21 [==============================] - 2s 114ms/step - loss: 1.3337 - accuracy: 0.4655 - val_loss: 1.4266 - val_accuracy: 0.4206
Epoch 5/10
21/21 [==============================] - 2s 114ms/step - loss: 1.2573 - accuracy: 0.4901 - val_loss: 1.4039 - val_accuracy: 0.4330
Epoch 6/10
21/21 [==============================] - 2s 114ms/step - loss: 1.2156 - accuracy: 0.5046 - val_loss: 1.4468 - val_accuracy: 0.4299
Epoch 7/10
21/21 [==============================] - 2s 114ms/step - loss: 1.1597 - accuracy: 0.5309 - val_loss: 1.5151 - val_accuracy: 0.3917
Epoch 

In [59]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 8005)              64088030  
                                                                 
 dropout_4 (Dropout)         (None, 8005)              0         
                                                                 
 dense_7 (Dense)             (None, 200)               1601200   
                                                                 
 dropout_5 (Dropout)         (None, 200)               0         
                                                                 
 dense_8 (Dense)             (None, 6)                 1206      
                                                                 
Total params: 65,690,436
Trainable params: 65,690,436
Non-trainable params: 0
_________________________________________________________________


In [60]:
# Estimating the accuracy on the test dataset using loaded weights
scores = model.evaluate(x_test_bow, y_test_six_way, verbose = 0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 41.70%


In [63]:
# Building the confusion matrix
from sklearn import metrics
y_pred = model.predict(x_test_bow)
metrics.confusion_matrix(y_test_six_way.argmax(axis = 1), y_pred.argmax(axis = 1))

array([[135,  14,  27,  36,  26,  12],
       [ 33,  37,  20,  51,  65,   5],
       [ 44,   4,  79,  55,  25,   7],
       [ 41,   9,  24, 132,  60,   1],
       [ 31,  24,  20,  64, 108,   2],
       [ 31,   0,   5,   8,   4,  44]])

### BERT

In [64]:
# Model fitting
model = create_model()
model.fit(x_train_bert, y_train_six_way, batch_size = batch_size, epochs = epochs, verbose = 1, validation_data = (x_val_bert, y_val_six_way))

Epoch 1/10
21/21 [==============================] - 3s 39ms/step - loss: 2.4698 - accuracy: 0.2193 - val_loss: 1.6615 - val_accuracy: 0.3263
Epoch 2/10
21/21 [==============================] - 0s 21ms/step - loss: 1.7014 - accuracy: 0.2836 - val_loss: 1.5978 - val_accuracy: 0.3232
Epoch 3/10
21/21 [==============================] - 0s 22ms/step - loss: 1.6423 - accuracy: 0.3203 - val_loss: 1.5612 - val_accuracy: 0.3544
Epoch 4/10
21/21 [==============================] - 0s 22ms/step - loss: 1.5800 - accuracy: 0.3452 - val_loss: 1.6410 - val_accuracy: 0.3466
Epoch 5/10
21/21 [==============================] - 0s 23ms/step - loss: 1.5530 - accuracy: 0.3656 - val_loss: 1.4219 - val_accuracy: 0.4346
Epoch 6/10
21/21 [==============================] - 0s 24ms/step - loss: 1.4943 - accuracy: 0.3912 - val_loss: 1.4519 - val_accuracy: 0.3995
Epoch 7/10
21/21 [==============================] - 0s 20ms/step - loss: 1.4610 - accuracy: 0.4069 - val_loss: 1.3966 - val_accuracy: 0.4252
Epoch 8/10
21

In [65]:
# Estimating the accuracy on the test dataset using loaded weights
scores = model.evaluate(x_test_bert, y_test_six_way, verbose = 0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 43.10%


In [66]:
# Building the confusion matrix
y_pred = model.predict(x_test_bert)
metrics.confusion_matrix(y_test_six_way.argmax(axis = 1), y_pred.argmax(axis = 1))

array([[140,   1,  33,  52,  15,   9],
       [ 41,  30,  25,  62,  49,   4],
       [ 42,   2,  86,  60,  19,   5],
       [ 40,   2,  32, 162,  31,   0],
       [ 28,  12,  24,  88,  95,   2],
       [ 32,   3,   8,   8,   1,  40]])

### GLOVE

In [67]:
# Model fitting
model = create_model()
model.fit(x_train_glove, y_train_six_way, batch_size = batch_size, epochs = epochs, verbose = 1, validation_data = (x_val_glove, y_val_six_way))

Epoch 1/10
21/21 [==============================] - 2s 35ms/step - loss: 2.5724 - accuracy: 0.2384 - val_loss: 1.6318 - val_accuracy: 0.3847
Epoch 2/10
21/21 [==============================] - 0s 20ms/step - loss: 1.6370 - accuracy: 0.3265 - val_loss: 1.4893 - val_accuracy: 0.4034
Epoch 3/10
21/21 [==============================] - 0s 18ms/step - loss: 1.5548 - accuracy: 0.3730 - val_loss: 1.4523 - val_accuracy: 0.4034
Epoch 4/10
21/21 [==============================] - 0s 18ms/step - loss: 1.4939 - accuracy: 0.3922 - val_loss: 1.3960 - val_accuracy: 0.4361
Epoch 5/10
21/21 [==============================] - 0s 19ms/step - loss: 1.4596 - accuracy: 0.4073 - val_loss: 1.3953 - val_accuracy: 0.4268
Epoch 6/10
21/21 [==============================] - 0s 18ms/step - loss: 1.4222 - accuracy: 0.4181 - val_loss: 1.3596 - val_accuracy: 0.4416
Epoch 7/10
21/21 [==============================] - 0s 19ms/step - loss: 1.3938 - accuracy: 0.4233 - val_loss: 1.3594 - val_accuracy: 0.4136
Epoch 8/10
21

In [68]:
# Estimating the accuracy on the test dataset using loaded weights
scores = model.evaluate(x_test_glove, y_test_six_way, verbose = 0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 44.12%


In [69]:
# Building the confusion matrix
y_pred = model.predict(x_test_glove)
metrics.confusion_matrix(y_test_six_way.argmax(axis = 1), y_pred.argmax(axis = 1))

array([[111,   1,  17,  80,  34,   7],
       [ 17,  34,  16,  64,  77,   3],
       [ 20,   0,  71,  78,  41,   4],
       [ 29,   3,  10, 155,  70,   0],
       [ 16,   0,   8,  72, 151,   2],
       [ 20,   0,   2,  18,   8,  44]])

# Binary Classification

## Preprocessing

In [70]:
num_classes = 2

In [71]:
# Function for preprocessing labels
def dataPreprocessingBinary(data):
    y = data["label"].tolist()

    # Changing the 'half-true', 'mostly-true', barely-true', 'pants-fire' labels to True/False for Binary Classification
    for x in range(len(y)):
        if(y[x] == 'half-true'):
            y[x] = 'True'
        elif(y[x] == 'mostly-true'):
            y[x] = 'True'
        elif(y[x] == 'barely-true'):
            y[x] = 'False'
        elif(y[x] == 'pants-fire'):
            y[x] = 'False'

    # Converting the lables into binary class matrix
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)
    y = np_utils.to_categorical(y, num_classes)
    return y

In [72]:
y_train_binary = dataPreprocessingBinary(train)
y_test_binary = dataPreprocessingBinary(test)
y_val_binary = dataPreprocessingBinary(val)

## Model
Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


In [73]:
## write your code here
# Initialize hyperparameters
# Create model
# train
# test
# report accuracy, f1-score and confusion matrix

In [74]:
## write your code here
# Initialize hyperparameters
learning_rate = 1e-3
epochs = 10
batch_size = 500
num_classes = 2
# train
# test
# report accuracy, f1-score and confusion matrix

In [75]:
def create_model():
  # Creating the model
  model = Sequential()
  model.add(Dense(8005, activation = 'relu', kernel_initializer = 'glorot_uniform'))
  model.add(Dropout(0.80))
  model.add(Dense(50, activation = 'relu'))
  model.add(Dropout(0.65))
  model.add(Dense(num_classes, activation = 'softmax'))
  rmsprop = RMSprop(learning_rate)
  model.compile(loss = 'categorical_crossentropy', optimizer = rmsprop, metrics = ['accuracy']) # Compile model
  return model

### BOW

In [76]:
# Model fitting
model = create_model()
model.fit(x_train_bow, y_train_binary, batch_size = batch_size, epochs = epochs, verbose = 1, validation_data = (x_val_bow, y_val_binary))

Epoch 1/10
21/21 [==============================] - 5s 155ms/step - loss: 0.7176 - accuracy: 0.6366 - val_loss: 0.5705 - val_accuracy: 0.7072
Epoch 2/10
21/21 [==============================] - 3s 125ms/step - loss: 0.5783 - accuracy: 0.7028 - val_loss: 0.5542 - val_accuracy: 0.7111
Epoch 3/10
21/21 [==============================] - 3s 126ms/step - loss: 0.5355 - accuracy: 0.7265 - val_loss: 0.5681 - val_accuracy: 0.6854
Epoch 4/10
21/21 [==============================] - 3s 126ms/step - loss: 0.5028 - accuracy: 0.7416 - val_loss: 0.5634 - val_accuracy: 0.7002
Epoch 5/10
21/21 [==============================] - 3s 124ms/step - loss: 0.4892 - accuracy: 0.7551 - val_loss: 0.5748 - val_accuracy: 0.6955
Epoch 6/10
21/21 [==============================] - 3s 120ms/step - loss: 0.4695 - accuracy: 0.7572 - val_loss: 0.6617 - val_accuracy: 0.6760
Epoch 7/10
21/21 [==============================] - 3s 121ms/step - loss: 0.4505 - accuracy: 0.7703 - val_loss: 0.6357 - val_accuracy: 0.6830
Epoch 

In [77]:
# Estimating the accuracy on the test dataset using loaded weights
scores = model.evaluate(x_test_bow, y_test_binary, verbose = 0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 70.23%


In [78]:
# Building the confusion matrix
y_pred = model.predict(x_test_bow)
metrics.confusion_matrix(y_test_binary.argmax(axis = 1), y_pred.argmax(axis = 1))

array([[333, 223],
       [159, 568]])

### BERT

In [79]:
# Model fitting
model = create_model()
model.fit(x_train_bert, y_train_binary, batch_size = batch_size, epochs = epochs, verbose = 1, validation_data = (x_val_bert, y_val_binary))

Epoch 1/10
21/21 [==============================] - 2s 27ms/step - loss: 0.7705 - accuracy: 0.5845 - val_loss: 0.6343 - val_accuracy: 0.6417
Epoch 2/10
21/21 [==============================] - 0s 20ms/step - loss: 0.6467 - accuracy: 0.6479 - val_loss: 0.6795 - val_accuracy: 0.5981
Epoch 3/10
21/21 [==============================] - 0s 18ms/step - loss: 0.6330 - accuracy: 0.6613 - val_loss: 0.5932 - val_accuracy: 0.6807
Epoch 4/10
21/21 [==============================] - 0s 22ms/step - loss: 0.6126 - accuracy: 0.6750 - val_loss: 0.6149 - val_accuracy: 0.6472
Epoch 5/10
21/21 [==============================] - 0s 19ms/step - loss: 0.6048 - accuracy: 0.6823 - val_loss: 0.5707 - val_accuracy: 0.6869
Epoch 6/10
21/21 [==============================] - 0s 21ms/step - loss: 0.5856 - accuracy: 0.6933 - val_loss: 0.5684 - val_accuracy: 0.6970
Epoch 7/10
21/21 [==============================] - 0s 19ms/step - loss: 0.5739 - accuracy: 0.7063 - val_loss: 0.5598 - val_accuracy: 0.7095
Epoch 8/10
21

In [80]:
# Estimating the accuracy on the test dataset using loaded weights
scores = model.evaluate(x_test_bert, y_test_binary, verbose = 0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 70.54%


In [81]:
# Building the confusion matrix
y_pred = model.predict(x_test_bert)
metrics.confusion_matrix(y_test_binary.argmax(axis = 1), y_pred.argmax(axis = 1))

array([[345, 211],
       [167, 560]])

### GLOVE

In [82]:
# Model fitting
model = create_model()
model.fit(x_train_glove, y_train_binary, batch_size = batch_size, epochs = epochs, verbose = 1, validation_data = (x_val_glove, y_val_binary))

Epoch 1/10
21/21 [==============================] - 1s 21ms/step - loss: 1.0400 - accuracy: 0.5911 - val_loss: 0.6421 - val_accuracy: 0.6121
Epoch 2/10
21/21 [==============================] - 0s 13ms/step - loss: 0.6419 - accuracy: 0.6417 - val_loss: 0.6181 - val_accuracy: 0.6316
Epoch 3/10
21/21 [==============================] - 0s 12ms/step - loss: 0.6241 - accuracy: 0.6662 - val_loss: 0.5958 - val_accuracy: 0.6799
Epoch 4/10
21/21 [==============================] - 0s 14ms/step - loss: 0.6072 - accuracy: 0.6762 - val_loss: 0.5508 - val_accuracy: 0.7165
Epoch 5/10
21/21 [==============================] - 0s 13ms/step - loss: 0.6041 - accuracy: 0.6812 - val_loss: 0.5465 - val_accuracy: 0.7173
Epoch 6/10
21/21 [==============================] - 0s 13ms/step - loss: 0.5937 - accuracy: 0.6921 - val_loss: 0.5329 - val_accuracy: 0.7087
Epoch 7/10
21/21 [==============================] - 0s 13ms/step - loss: 0.5844 - accuracy: 0.6985 - val_loss: 0.5483 - val_accuracy: 0.7111
Epoch 8/10
21

In [83]:
# Estimating the accuracy on the test dataset using loaded weights
scores = model.evaluate(x_test_glove, y_test_binary, verbose = 0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 72.72%


In [84]:
# Building the confusion matrix
y_pred = model.predict(x_test_glove)
metrics.confusion_matrix(y_test_binary.argmax(axis = 1), y_pred.argmax(axis = 1))

array([[319, 237],
       [113, 614]])